# README

*   코랩을 기준으로 코드를 작성했습니다.
*   로직은 마크다운의 순서로 보시면 됩니다.
*   사용자 사전은 ***'정규표현식'*** 파트와 ***'단어사전 추가'*** 파트에 있습니다.
*   ***'데이터 불러오기'*** 쉘에서 각 쇼핑몰 데이터 파일에 대한 경로를 수정해주시면 됩니다.
*   모든 쉘을 실행시킵니다.
*   ***'결과물 출력'*** 쉘을 실행해서 메디스트림 상품명을 입력하시면 각 쇼핑몰마다 유사한 상품을 결과로 출력합니다.

예시) 메디스트림의 상품명 '[진산] 척추모형'을 '결과물 출력' 쉘을 실행한 뒤 해당 내용을 입력하면 타사 쇼핑몰에서 자사 상품과 유사한 상품을 출력합니다.


# 드라이브 마운트 및 import

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pip install customized_konlpy
from ckonlpy.tag import Twitter
twitter = Twitter()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [8]:
import time
start = time.time()

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numpy import dot
from numpy.linalg import norm
import json
import re
from pprint import pprint
from math import log # IDF 계산을 위해

# TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer

# 데이터 불러오기

In [9]:
# 데이터 불러오기

# 아콤몰 데이터 불러오기
df1 = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data/jsakommall_t.json')
# 안진팜메디 데이터 불러오기
df2 = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data/jsanjinmed_t.json')
# 한의몰 데이터 불러오기
df3 = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data/jshanemall_t.json')
# 한퓨어몰 데이터 불러오기
df4 = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data/jshanpuremall_t.json')
# 휴먼허브 데이터 불러오기
df5 = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data/jshumanherb_t.json')
# 케이엠몰 데이터 불러오기
df6 = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data/jskmmall_t.json')
# 대중메디팜 데이터 불러오기
df7 = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data/jsoneshop_t.json')

# 메디 스트림 데이터 불러오기
products = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data/products.json')

# 정답셋
answer = pd.read_json('/content/drive/MyDrive/Colab Notebooks/이어드림 프로젝트/기업연계 프로젝트/코드/answer.json')

# 전처리

## 중복 제거

In [10]:
# 메디스트림
medistream_columns = ['_id','name', 'sale_price']
medistream = products.drop_duplicates(['name']).reset_index(drop=True)
medistream = medistream.loc[:, medistream_columns]

# 아콤몰
others_columns = ['market_name', 'name', 'price', 'id', 'url']
akom = df1.drop_duplicates(['name']).reset_index(drop=True)
akom = akom.loc[:,others_columns]

# 안진팜메디
anjin = df2.drop_duplicates(['name']).reset_index(drop=True)
anjin = anjin.loc[:, others_columns]

# 한의몰
hane = df3.drop_duplicates(['name']).reset_index(drop=True)
hane = hane.loc[:, others_columns]

# 한퓨어몰
hanpure = df4.drop_duplicates(['name']).reset_index(drop=True)
hanpure = hanpure.loc[:, others_columns]

# 휴먼허브
human_hub = df5.drop_duplicates(['name']).reset_index(drop=True)
human_hub = human_hub.loc[:, others_columns]

# 케이엠몰
km_mall = df6.drop_duplicates(['name']).reset_index(drop=True)
km_mall = km_mall.loc[:, others_columns]

# 대중메디팜
daejoong = df7.drop_duplicates(['name']).reset_index(drop=True)
daejoong = daejoong.loc[:, others_columns]

## 컬럼 리스트화

In [11]:
# 메디스트림
medistream_id_list = list(medistream['_id'])
medistream_name_list = list(medistream['name'])
medistream_price_list = list(medistream['sale_price'])

# 아콤몰
akom_market_name_list = list(akom['market_name'])
akom_name_list = list(akom['name'])
akom_price_list = list(akom['price'])
akom_id_list = list(akom['id'])
akom_url_list = list(akom['url'])

# 안진팜메디
anjin_market_name_list = list(anjin['market_name'])
anjin_name_list = list(anjin['name'])
anjin_price_list = list(anjin['price'])
anjin_id_list = list(anjin['id'])
anjin_url_list = list(anjin['url'])

# 한의몰
hane_market_name_list = list(hane['market_name'])
hane_name_list = list(hane['name'])
hane_price_list = list(hane['price'])
hane_id_list = list(hane['id'])
hane_url_list = list(hane['url'])

# 한퓨어몰
hanpure_market_name_list = list(hanpure['market_name'])
hanpure_name_list = list(hanpure['name'])
hanpure_price_list = list(hanpure['price'])
hanpure_id_list = list(hanpure['id'])
hanpure_url_list = list(hanpure['url'])

# 휴먼허브
human_hub_market_name_list = list(human_hub['market_name'])
human_hub_name_list = list(human_hub['name'])
human_hub_price_list = list(human_hub['price'])
human_hub_id_list = list(human_hub['id'])
human_hub_url_list = list(human_hub['url'])

# 케이엠몰
km_mall_market_name_list = list(km_mall['market_name'])
km_mall_name_list = list(km_mall['name'])
km_mall_price_list = list(km_mall['price'])
km_mall_id_list = list(km_mall['id'])
km_mall_url_list = list(km_mall['url'])

# 대중메디팜
daejoong_market_name_list = list(daejoong['market_name'])
daejoong_name_list = list(daejoong['name'])
daejoong_price_list = list(daejoong['price'])
daejoong_id_list = list(daejoong['id'])
daejoong_url_list = list(daejoong['url'])

## 정규표현식

In [12]:
# 메디스트림 정규표현식
medistream_rex_list = []

for i in range(len(medistream_name_list)):
    # 공통 정규표현식
    medistream_rex = re.sub(r',0' , '0',  medistream_name_list[i])

    # 영어 대문자를 소문자로 변경
    medistream_rex = re.sub("[A-Z]", lambda x : x.group().lower(),medistream_rex)

    # 단어 변환
    medistream_rex = re.sub(r"maxbelt" , ' 맥스벨트 ',medistream_rex)
    medistream_rex = re.sub(r"active" , ' 액티브 ',medistream_rex)
    medistream_rex = re.sub(r"roll" , ' 롤 ',medistream_rex)
    medistream_rex = re.sub(r"aerobic" , ' 에어로빅 ',medistream_rex)
    medistream_rex = re.sub(r"stepbox" , ' 스텝박스 ',medistream_rex)
    medistream_rex = re.sub(r"massage" , ' 마사지 ',medistream_rex)
    medistream_rex = re.sub(r"bed" , ' 베드 ',medistream_rex)
    medistream_rex = re.sub(r"inbody" , ' 인바디 ',medistream_rex)
    medistream_rex = re.sub(r"thera" , ' 세라 ',medistream_rex)
    medistream_rex = re.sub(r"쎄라" , ' 세라 ',medistream_rex)
    medistream_rex = re.sub(r"band" , ' 밴드 ',medistream_rex)
    medistream_rex = re.sub(r"free" , ' 프리 ',medistream_rex)
    medistream_rex = re.sub(r"bond" , ' 본드 ',medistream_rex)
    medistream_rex = re.sub(r"동인" , ' 경혈인형 ',medistream_rex)
    medistream_rex = re.sub(r"大" , ' 대大 ',medistream_rex)
    medistream_rex = re.sub(r"中" , ' 중中 ',medistream_rex)
    medistream_rex = re.sub(r"小" , ' 소小 ',medistream_rex)
    medistream_rex = re.sub(r"lanzo" , ' 란조 ',medistream_rex)
    medistream_rex = re.sub(r"gp" , ' 굿플 ',medistream_rex)
    medistream_rex = re.sub(r"vials" , ' 바이알 ',medistream_rex)
    medistream_rex = re.sub(r"vial" , ' 바이알 ',medistream_rex)
    medistream_rex = re.sub(r"50일분" , ' 50일',medistream_rex)
    medistream_rex = re.sub(r"50일자포" , '50일',medistream_rex)
    medistream_rex = re.sub(r"raphael" , ' 라파엘 ',medistream_rex)
    medistream_rex = re.sub(r"db" , ' 닥터백 ',medistream_rex)
    medistream_rex = re.sub(r"알콜" , ' 알코올 ',medistream_rex)
    medistream_rex = re.sub(r"moa" , ' 모아 ',medistream_rex)
    medistream_rex = re.sub(r"ultragrip" , ' 울트라그립 ',medistream_rex)
    medistream_rex = re.sub(r"sft" , ' 세이프터치 ',medistream_rex)
    medistream_rex = re.sub(r"latex" , ' 라텍스 ',medistream_rex)
    medistream_rex = re.sub(r"glove" , ' 글로브 ',medistream_rex)
    medistream_rex = re.sub(r"needle" , ' 니들 ',medistream_rex)
    medistream_rex = re.sub(r"bd" , ' 백톤디킨슨 ',medistream_rex)
    medistream_rex = re.sub(r"ast/alt" , ' 간기능 ',medistream_rex)
    medistream_rex = re.sub(r"tens" , ' 텐스 ',medistream_rex)
    medistream_rex = re.sub(r"banshin" , ' 반신 ',medistream_rex)
    medistream_rex = re.sub(r"스폰지" , ' 스펀지 ',medistream_rex)
    medistream_rex = re.sub(r"patch" , ' 패치 ',medistream_rex)
    medistream_rex = re.sub(r"cettum" , ' 새뜸 ',medistream_rex)
    medistream_rex = re.sub(r"2 520" , ' 2520 ',medistream_rex)
    medistream_rex = re.sub(r"e뜸" , '무연전자 왕뜸기 e뜸',medistream_rex)
    

    # 불용어
    medistream_rex = re.sub(r"(3b)|[0-9][0-9]\.|기본사이즈|분리|파트|포함|의료|기기|본사|협약|판매|medistream|members", '',medistream_rex)
    medistream_rex = re.sub(r"전용몰|런칭|기념|묶음|진산|eu|생산|유럽|척추스탠드|pcs|다용도|HCK|세트|전용|1개", '',medistream_rex)
    medistream_rex = re.sub(r"아이보리|r60200|r3060|r3035|개입|1갑|amp|ea|가 있는", '',medistream_rex)
    medistream_rex = re.sub(r"1박스|1회|용량|[0-9].[0-9][0-9]g|[0-9]단계|2정|6ea|(1527-2)|(1583)|(1581)|1팩", '',medistream_rex)
    medistream_rex = re.sub(r"클래식|veri(50mm*9.1m)|(6ea/box)|3(7.5cm*4.5m)|(24ea/box)|1(2.5cm*4.5m)|(30ea/box) (1581)", '',medistream_rex)
    medistream_rex = re.sub(r"(588-30)|(582-10)|(25mm*9.1m)|(12ea/box)|(1530-1)|실속형|(4.5cm×4.5cm)|12박스", '',medistream_rex)
    medistream_rex = re.sub(r"기본사이즈|10박스|pp|130|ez|낱개|세트|(4극+4극)|1세트|\(cy[^)]*\)|1kg|47*36(cm)|22mm|직접|구용|이 있는|박스", '',medistream_rex)

    # 특수문자 제거
    medistream_rex = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", " ", medistream_rex)
    medistream_rex = re.sub(r"\[|\]|\(|\)|\<|\>", " ", medistream_rex)

    medistream_rex_list.append(medistream_rex)

In [13]:
# 타사 전처리
akom_rex_list = []
for i in range(len(akom_name_list)):
    # 공통 정규표현식
    
    akom_rex = re.sub(r',0' , '0', akom_name_list[i])
    # 영어 대문자를 소문자로 변경
    akom_rex = re.sub("[A-Z]", lambda x : x.group().lower(),akom_rex)
    # 단어 변환
    akom_rex = re.sub(r"大" , ' 대大 ',akom_rex)
    akom_rex = re.sub(r"中" , ' 중中 ',akom_rex)
    akom_rex = re.sub(r"小" , ' 소小 ',akom_rex)
    akom_rex = re.sub(r"dressing" , ' 드레싱 ',akom_rex)
    akom_rex = re.sub(r"forcep" , ' 포셉 ',akom_rex)
    akom_rex = re.sub(r"벅 해머" , ' 벅해머 ',akom_rex)
    akom_rex = re.sub(r"mono" , ' 모노 ',akom_rex)
    akom_rex = re.sub(r"클로르페니라민말레산염" , ' 클로르페나라민말레산염 ',akom_rex)
    akom_rex = re.sub(r"inbody" , ' 인바디 ',akom_rex)
    akom_rex = re.sub(r"banshin" , ' 반신 ',akom_rex)
    akom_rex = re.sub(r"micro" , ' 마이크로 ',akom_rex)
    akom_rex = re.sub(r"needle" , ' 니들 ',akom_rex)
    akom_rex = re.sub(r"solar" , ' 쏠라 ',akom_rex)
    akom_rex = re.sub(r"lay" , ' 레이 ',akom_rex)
    akom_rex = re.sub(r"micropore" , ' 마이크로포 ',akom_rex)
    akom_rex = re.sub(r"anylox" , ' 애니록스 ',akom_rex)
    akom_rex = re.sub(r"bd" , ' 백톤디킨슨 ',akom_rex)
    akom_rex = re.sub(r"삼릉" , '삼능',akom_rex)
    akom_rex = re.sub(r"필텍" , '필젯',akom_rex)

    # 불용어
    akom_rex = re.sub(r"2인용|보험|청구|가능밀착포|원진|허브원|1팩|pcs|pvc|선월드|코리아|맘모스|계량컵|파키스탄|전체|a형|은색|gmp|인증|손질|세척|절단|상품|pcs|이비인후과|ear|", '',akom_rex)
    akom_rex = re.sub(r"ubiomacpa|보라색|전문가용|2인용|냉장|시약|세트|셀바스|(구)|자원메디칼|신상품|삼성프린터기/테이블포함|65℃|전용|비만|자동엑티베이터건|터보|싸소|full|option|리엔더테이블|원터치|커프|출력|안됨|전용|set|16,000cc용|면역|분석|", '',akom_rex)
    akom_rex = re.sub(r"box|포장|\(100[^)]*증정\)|성광제약|ea|1/2인치|살색|", '',akom_rex)
    akom_rex = re.sub(r"멘토|생화학|16000cc용|최신|제조품|백색|½|선월드|코리아|어댑터포함|포함|바늘없음|60mm|5/16|증정|", '',akom_rex)

    # 특수문자 제거
    akom_rex = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", " ", akom_rex)
    akom_rex = re.sub(r"\[|\]|\(|\)|\<|\>", " ", akom_rex)

    akom_rex_list.append(akom_rex)

anjin_rex_list = []
for i in range(len(anjin_name_list)):
    # 공통 정규표현식
    anjin_rex = re.sub(r',0' , '0', anjin_name_list[i])
    # 영어 대문자를 소문자로 변경
    anjin_rex = re.sub("[A-Z]", lambda x : x.group().lower(),anjin_rex)
    # 단어 변환
    anjin_rex = re.sub(r"大" , ' 대大 ',anjin_rex)
    anjin_rex = re.sub(r"中" , ' 중中 ',anjin_rex)
    anjin_rex = re.sub(r"小" , ' 소小 ',anjin_rex)
    anjin_rex = re.sub(r"쎄라" , ' 세라 ',anjin_rex)
    anjin_rex = re.sub(r"큐" , ' q ',anjin_rex)
    anjin_rex = re.sub(r"핫백" , ' 핫팩 ',anjin_rex)
    anjin_rex = re.sub(r"inbody" , ' 인바디 ',anjin_rex)
    anjin_rex = re.sub(r"banshin" , ' 반신 ',anjin_rex)
    anjin_rex = re.sub(r"gd" , ' 굿플 ',anjin_rex)
    anjin_rex = re.sub(r"biofreeze" , ' 바이오프리즈 ',anjin_rex)
    anjin_rex = re.sub(r"필젝" , ' 필젯 ',anjin_rex)
    # 불용어
    anjin_rex = re.sub(r"\(100봉[^)]*pcs\)|eu|응급의약품|일반의약품|3b모형|독일|생산|가 있는|운동형|포함| 및 |변화|설명|부분채색", '',anjin_rex)
    anjin_rex = re.sub(r"쿠폰|사용|불가|10갑|손잡이|국내|생산|의약외품|opp|이 있는|sam최상위모델|실제인체크기|176.5cm/10.35kg|인대와 근육이 부착 표현된", '',anjin_rex)
    anjin_rex = re.sub(r"살색|흰색지름6.0cm/물흡수시 약6.5cm|b타입|납품|병원|매수가 변경되었습니다|도자선|세트|선택|도자컵|리드선|수+수|신우|정형용|교정|장치|인천|단순|변심|교환|반품|불가|접이형|전문가용|b형|용량|예약타이머|공장|직송|몸통:스텐리스|2인용|서울|경기|수도권|구매|가능|전화|문의|토기|시어스제약|\(정제[^)]*약\)|\(연조엑스[^)]*약\)|\(정제[^)]*품\)|\(산제[^)]*약\)|\(연조엑스[^)]*제\)|\(연조엑스[^)]*품\)|\(정제[^)]*제\)|\(산제[^)]*제\)|\(산제[^)]*품\)|\(4.[^)]*포\)|마일리지|\(단[^)]*약\)|한방건강|보험용|신약|gcjbp|염산염수화물|1갑|백색|원료의약품|외피용 살균 소독제|\(3.[^)]*포\)|\(2.[^)]*포\)|k-[0-9][0-9]|(3.0g×100포)|\[보험[^)]*\]|한방병|의원|전용|\●2[^)]*\●|협심증,관상동맥경화 치료경감을 위한 생약제|전문의약품|일반의약품|2mlx50앰플|1회만|증정|id당|인태반 유래-갱년기 증상 개선 주사제|전문의약품|1갑|발수성|테이핑테이프/헝겊절단/불소코팅|ns-160|하세가와|프라스틱|3단|2단|4단|길이|pcs|4묶음x100장|55×55mm|ykhc|아주가늘다|1갑|트리코트|폴리에스터|pvc|sbc|스웨이드|크기|sun|\(a[0-9][^)]*\)|옷칠|함유|공용|유구|의약외품|boric|안진약품|sp-7503|안진팜메디|직수입|\#여름[^)]*\#|인쇄|무인쇄|\(세트[^)]*\)|지름|기성|인쇄|뚜껑|세트|구성|한신제약|의약외품|신경과용|kawe|일반용|끝에 길이 50m 인쇄되어 있습니다|국내|박스|약47×52cm|이비인후과용|신기구|사용|가능|덕용|30G- 25mm|길이|pcs|트레이frcep|jar|\(cy[^)]*\)|dressing|", '',anjin_rex)
    anjin_rex = re.sub(r"\(두께[^)]*\)|약포지|(0.30×40/60)|감마선멸균침|\(0.40[^)]*[0-9]\)|\(두[^)]*[0-9]\)|감마선|md|포장|무독성|살균제|안전성|친환경적|식약청|허가|제품|", '',anjin_rex)
    anjin_rex = re.sub(r"단타|쏠라세이프|연타|겸용|진료용|무분말|주문|1과|½|인치|\[p[0-9][^)]*[0-9]\]|지름6.0cm|혋부|항시|사용|100개씩소분포장불가|", '',anjin_rex)
    anjin_rex = re.sub(r"사은품|텍사메타손포스페이트이나트륨|앰플|플라스틱손잡이|외피용|살균소독제|흡착탈지면|이소프로판올|50봉x20pcs|가스멸균|\(0[^)]*0\)|\#사[^)]*정\#|약44개|power|free|사이즈|(xs,s,m,l)|켤레|1호,2호,3호,4호,5호|(EO가스멸균)|벌크|소분", '',anjin_rex)
    # 특수문자 제거
    anjin_rex = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", " ", anjin_rex)
    anjin_rex = re.sub(r"\[|\]|\(|\)|\<|\>", " ", anjin_rex)
    
    anjin_rex_list.append(anjin_rex)

hane_rex_list = []
for i in range(len(hane_name_list)):
    # 공통 정규표현식
    hane_rex = re.sub(r',0' , '0', hane_name_list[i])
    # 영어 대문자를 소문자로 변경
    hane_rex = re.sub("[A-Z]", lambda x : x.group().lower(),hane_rex)
    # 단어 변환
    hane_rex = re.sub(r"大" , ' 대大 ',hane_rex)
    hane_rex = re.sub(r"中" , ' 중中 ',hane_rex)
    hane_rex = re.sub(r"小" , ' 소小 ',hane_rex)
    hane_rex = re.sub(r"비커" , ' 비이커 ',hane_rex)
    hane_rex = re.sub(r"airone" , ' 에어원 ',hane_rex)
    hane_rex = re.sub(r"흡인" , ' 흡입 ',hane_rex)
    hane_rex = re.sub(r"simplex" , ' 심플렉스 ',hane_rex)
    hane_rex = re.sub(r"tas" , ' 타스 ',hane_rex)
    hane_rex = re.sub(r"유닛" , ' 유니트 ',hane_rex)
    hane_rex = re.sub(r"remed" , ' 리메드 ',hane_rex)
    hane_rex = re.sub(r"pioneer" , ' 파이오니어 ',hane_rex)
    hane_rex = re.sub(r"간섭전류형" , ' ict ',hane_rex)
    hane_rex = re.sub(r"\[db\]" , ' 닥터백 ',hane_rex)
    hane_rex = re.sub(r"\[bd\]" , ' 백톤디킨슨 ',hane_rex)
    hane_rex = re.sub(r"\(db\)" , ' 닥터백 ',hane_rex)
    hane_rex = re.sub(r"long" , ' 롱 ',hane_rex)
    hane_rex = re.sub(r"short" , ' 숏 ',hane_rex)
    hane_rex = re.sub(r"type" , ' 타입 ',hane_rex)
    hane_rex = re.sub(r"수퍼" , ' 슈퍼 ',hane_rex)
    # 불용어
    hane_rex = re.sub(r"1box|개입|박스|한쪽|단위|1개|길이|색상|6cm|폭 3cm|단면|전체금|한의|", '',hane_rex)
    hane_rex = re.sub(r"플라스틱|15cm|지름|높이|생명사랑|전용|1팩|ea|사은품|증정|기능", '',hane_rex)
    hane_rex = re.sub(r"600x2000mm|1롤|롤형|2단|3단|단위|1통|세트|의료보험|청구|보험", '',hane_rex)
    hane_rex = re.sub(r"1750개|가능|poct|기기|5620|성인용|2인용|promed|4개|1조", '',hane_rex)
    hane_rex = re.sub(r"정품|6개|전문가용|충c격파|2인용|낱개|mts|천연펄프|내과|질환|청구가|청구", '',hane_rex)
    hane_rex = re.sub(r"넥스케어|살색종이|흰색종이20p|0.3/0.5/1cc|30g,31|100조|(위생방수지)", '',hane_rex)
    hane_rex = re.sub(r"20pcs|0.30mm|60mm|5개|(2*2*8p)|(4.5cm*4.5cm*8p)|1박스|27g60mm|30g*1/2|30g*38mm|31g", '',hane_rex)
    hane_rex = re.sub(r"5/16|1박스|사은품|증정|택1|국내산|10박스|포장|규격|0.18*8mm|(0.5/0.6x50mm,80mm)", '',hane_rex)
    # 특수문자 제거
    hane_rex = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", " ", hane_rex)
    hane_rex = re.sub(r"\[|\]|\(|\)|\<|\>", " ", hane_rex)

    hane_rex_list.append(hane_rex)

km_mall_rex_list = []
for i in range(len(km_mall_name_list)):
    # 공통 정규표현식
    km_mall_rex = re.sub(r',0' , '0', km_mall_name_list[i])
    # 영어 대문자를 소문자로 변경
    km_mall_rex = re.sub("[A-Z]", lambda x : x.group().lower(),km_mall_rex)
    # 단어 변환
    km_mall_rex = re.sub(r"大" , ' 대大 ',km_mall_rex)
    km_mall_rex = re.sub(r"中" , ' 중中 ',km_mall_rex)
    km_mall_rex = re.sub(r"小" , ' 소小 ',km_mall_rex)
    km_mall_rex = re.sub(r"lancets" , ' 란셋 ',km_mall_rex)
    km_mall_rex = re.sub(r"gp" , ' 굿플 ',km_mall_rex)
    km_mall_rex = re.sub(r"inbody" , ' 인바디 ',km_mall_rex)
    km_mall_rex = re.sub(r"스탠딩" , ' 스탠드 ',km_mall_rex)
    km_mall_rex = re.sub(r"banshin" , ' 반신 ',km_mall_rex)
    km_mall_rex = re.sub(r"st" , ' 스트라텍 ',km_mall_rex)
    km_mall_rex = re.sub(r"1만쌈" , ' 10000 ',km_mall_rex)
    km_mall_rex = re.sub(r"bd" , ' 백톤디킨슨 ',km_mall_rex)
    km_mall_rex = re.sub(r"long" , ' 롱 ',km_mall_rex)
    km_mall_rex = re.sub(r"short" , ' 숏 ',km_mall_rex)
    km_mall_rex = re.sub(r"type" , ' 타입 ',km_mall_rex)
    km_mall_rex = re.sub(r"필텍" , ' 필젯 ',km_mall_rex)
    km_mall_rex = re.sub(r"mono" , ' 모노 ',km_mall_rex)
    # 불용어
    km_mall_rex = re.sub(r"뜸그릇|대덕|전용|정품|경희|국내|생산|new|메디렉스|런칭|특가|핫솔루션|의료기기|", '',km_mall_rex)
    km_mall_rex = re.sub(r"pcs|4등급||10팩|할인|불가|상품|열린|3단카트|2단카트|(b2)|set|의료용|제조사|정품", '',km_mall_rex)
    km_mall_rex = re.sub(r"전문가용|추나베드|카이로테이블|풀옵션|납품|교환|반품|전기식 온구기 / 전자번침(온도 65도)", '',km_mall_rex)
    km_mall_rex = re.sub(r"정품|2인용|4p|1조|c2|(43도&45도)|묶음|선월드코리아|거치대|의약외품", '',km_mall_rex)
    km_mall_rex = re.sub(r"1갑|50팩|접수증|증정|20t|31g|1카톤|red|230v|e27|es|대한|(5cm x 5cm)", '',km_mall_rex)
    km_mall_rex = re.sub(r",400개입|1carton|1box|(1200조,4800개입)|1호,2호용|1카톤|8box|성호|31G 1.0|5/16(8mm)", '',km_mall_rex)
    km_mall_rex = re.sub(r"보성|특가|동화|1cc 31GX8mm, 5/16|29g|은도금|10통|Pro70 6box|네오|닥터|대량|특가|", '',km_mall_rex)
    km_mall_rex = re.sub(r"포장|개입|케어|1cc 31GX8mm, 5/16|(Normal Cap)|box", '',km_mall_rex)
    # 특수문자 제거
    km_mall_rex = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", " ", km_mall_rex)
    km_mall_rex = re.sub(r"\[|\]|\(|\)|\<|\>", " ", km_mall_rex)

    km_mall_rex_list.append(km_mall_rex)

daejoong_rex_list = []
for i in range(len(daejoong_name_list)):
    # 공통 정규표현식
    daejoong_rex = re.sub(r',0' , '0', daejoong_name_list[i])
    # 영어 대문자를 소문자로 변경
    daejoong_rex = re.sub("[A-Z]", lambda x : x.group().lower(),daejoong_rex)
    # 단어 변환
    daejoong_rex = re.sub(r"大" , ' 대大 ',daejoong_rex)
    daejoong_rex = re.sub(r"中" , ' 중中 ',daejoong_rex)
    daejoong_rex = re.sub(r"小" , ' 소小 ',daejoong_rex)
    daejoong_rex = re.sub(r"동인" , ' 경혈인형 ',daejoong_rex)
    daejoong_rex = re.sub(r"핫백" , ' 핫팩 ',daejoong_rex)
    daejoong_rex = re.sub(r"infralux" , 'ir',daejoong_rex)
    daejoong_rex = re.sub(r"클로르페니라민말레산염주" , ' 클로르페나라민말레산염 ',daejoong_rex)
    daejoong_rex = re.sub(r"간섭전류형" , ' ict ',daejoong_rex)
    daejoong_rex = re.sub(r"gp" , ' 굿플 ',daejoong_rex)
    daejoong_rex = re.sub(r"Dr.Back" , ' 닥터백 ',daejoong_rex)
    daejoong_rex = re.sub(r"카바" , ' 커버 ',daejoong_rex)
    daejoong_rex = re.sub(r"bd" , ' 백톤디킨슨 ',daejoong_rex)
    daejoong_rex = re.sub(r"필텍" , ' 필젯 ',daejoong_rex)
    # 불용어
    daejoong_rex = re.sub(r"중국제|82cm|3b|이비인후과|은색|30개|1박스|매입|10갑", '',daejoong_rex)
    daejoong_rex = re.sub(r"5.5 x 5.5cm|쿠폰|사은품|불가|기능|옐로우|레벨1|배너|병의원당|1회|증정", '',daejoong_rex)
    daejoong_rex = re.sub(r"pcs|ea|유효|기간|2024.10.04|응급의약품|단미엑스혼합제|ㄱ자형／자석형", '',daejoong_rex)
    daejoong_rex = re.sub(r"pop|무료|증정|전용|1200장(가로2cm*세로3cm)|set|드레싱카201a 1대 포함|전문가용", '',daejoong_rex)
    daejoong_rex = re.sub(r"본체|없이|사용|불가|별도|4채널|살색|흰색|브라운|화이트|은나노수 55ml", '',daejoong_rex)
    daejoong_rex = re.sub(r"½인치|50팩|8박스|(26g / 27g)|30g 1½|인치|박스|주사기x|31g * 8mm|31g * 8mm", '',daejoong_rex)
    # 특수문자 제거
    daejoong_rex = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", " ", daejoong_rex)
    daejoong_rex = re.sub(r"\[|\]|\(|\)|\<|\>", " ", daejoong_rex)

    daejoong_rex_list.append(daejoong_rex)


# 토큰화

## 전처리한 상품명 리스트화 및 합치기

In [14]:
# 상품명 리스트 합치기
# 그냥 set_list에서 연결하자
medistream_akom_name_list = medistream_rex_list + akom_rex_list
medistream_anjin_name_list= medistream_rex_list + anjin_rex_list 
medistream_hane_name_list = medistream_rex_list + hane_rex_list 
medistream_hanpure_name_list = medistream_rex_list + hanpure_name_list 
medistream_humna_hub_name_list = medistream_rex_list + human_hub_name_list 
medistream_km_mall_name_list = medistream_rex_list + km_mall_rex_list 
medistream_daejoong_name_list = medistream_rex_list + daejoong_rex_list

# twitter


## 단어사전 추가

In [15]:
# 단어사전 리스트
bnm = ['3B','3M','Callidesign','ERLERZIMMER','NASCO','REMED','SMC','SPIRIT','WELLBE','경희테크','계림','광덕','굿플','그린','그린란','기성','길우','나보소','네오스템','넥스코','녹십자웰빙','녹원','농림생약','다온','닥터백','대건','대경','대명','대신코스코','대연','대요메디','대중','대한','대한위재','대한한방','도도텍','동방','동양','드렉텍','디에스메디','라이넥주','리메드','리트만','메디스트림','메디스트림스탠다드','바른','바이오메드','삼릉종가','삼부','생명사랑','선월드코리아','성광','성심','세이버플러스','세이프란','세이프씰','수성','신흥제약','쏠라','쏠라레이','씨케이','씨케이주식회사','아콤','안동농협','안진팜메디','알피니언','에듀클린','에어원','에프론','에프에이','엔케어','엔탭','엠앤엠','영인','영일엠','온누리','용보','우전','원창','월드','위드라이너','유니메드','유니쎈타주','유한킴벌리','이도','인텍스','일라이트','자담생약','자은생약','절판','정우','제일','제트바이오텍','지하이웰','진산','진수','참조은','천양사','크라시에','키르키','타스컴','타원형','토니켓','트리셀','특수매선','파워닷','퍼슨','필립스','한동허브','한솔','한의','한의사랑','한일','함소아','행림','현대','현대교역','현대메디텍','현미온미','현진','휴먼','휴온스']
op1 = ['100000pcs','100000쌈','10000pcs','10000쌈','1000pcs','1000개','1000매','1000쌈','100cc','100cm','100pcs','100개','100개입','100매','100박스','100쌈','100장','100조','100팩','100포','10EA','10PCS','10T','10box','10cc','10cm','10ea','10pack','10pcs','10갑','10개','10개입','10단','10매','10매입','10묶음','10박스','10장입','10조','10팩','10포','10환','10환용','114cm','120cm','125매','12ea','12개','12단','12롤','12박스','12팩','137cm','150포','150환','15cm','15개','16cm','16oz','17cm','1800cm','180pcs','180개입','18cm','18ea','18개','18팩','1EA','1box','1ea','1kg','1pack','1pcs','1갑','1개','1묶음','1박스','1봉','1장','1조','1통','1팩','1포','1호','2000cc','2000pcs','200cm','200pcs','200개입','200매','200입','20T','20cm','20갑','20개','20롤','20매입','20박스','20통','20팩','210cm','220cm','22pcs','24cm','24ea','24롤','24팩','250pcs','250개입','250매','2520매','25PCS','25T','25cm','25개','2700쌈','2EA','2P','2box','2ea','2kg','2pack','2pcs','2갑','2개','2단','2롤','2매','2매입','2박스','2스틱','2절','2팩','2포','2호','30000pcs','3000cc','3000pcs','3000쌈','300cc','300cm','300매','300쌈','300포','30box','30cm','30ea','30개입','30통','30팩','30포','3600pcs','36cm','36ea','36개입','36매','36팩','38cm','3EA','3box','3cc','3ea','3oz','3pack','3pcs','3갑','3개','3단','3박스','3팩','3포','3호','400매','400장','40PCS','40cm','40ea','40개','40롤','40매','40박스','43cm','44cm','4500쌈','45cm','45팩','48개입','48매','48팩','4EA','4box','4cm','4ea','4oz','4p','4pack','4pcs','4갑','4개','4개입','4겹','4박스','4절','4팩','4포','4호','50000pcs','50000쌈','5000pcs','5000쌈','500개입','500매','500쌈','500장','500환','50PCS','50cm','50pcs','50개','50개입','50매','50박스','50통','50팩','51cm','54cm','5EA','5PCS','5box','5cc','5cm','5ea','5kg','5pack','5pcs','5갑','5개','5개입','5묶음','5박스','5장','5팩','5포','5호','600매','600장','60cm','60박스','60팩','6cm','6ea','6개','6도','6롤','6매','6박스','6절','6포','6호','70매','75cm','75매','76cm','7cm','7절','7호','80cm','85cm','8P','8겹','8단','8롤','8절','8팩','9000쌈','900개입','900쌈','90cm','91cm','96팩','9cm','9포','9호']
op2 = ['0.5kg','0.5mL','0.8mm','1.26g','1.2kg','1.2mm','1.41g','1.61g','1.6mm','1000g','100cc','100cm','100g','100mL','100ml','101G','10cc','10cm','10g','10m','10mL','114cm','115g','11mm','12.5mm','120cm','120g','120mm','12g','12mm','135mm','137cm','140L','14cm','150g','150mm','15cm','15g','15mL','15mm','160g','16cm','17cm','17mm','1800cm','18G','18L','18cm','1L','1g','1kg','1mL','2,000cc','2.18g','2.42g','2.48g','2.52g','2.5cm','2.66g','2.67g','2.86g','2.89g','2.91g','2.92g','2004G','200cm','200g','200ml','201G','20cm','20cut','20g','20mL','210cm','220cm','22L','22mm','23G','24mm','250ml','25L','25cm','25ml','25mm','26G','26g','280ml','28G','2L','2mL','3.02g','3.0g','3.19g','3.26g','3.27g','3.36g','3.39g','3.44g','3.59g','3.5cm','3.65g','3.7cm','3000cc','300L','300c','300cc','300g','301G','302G','30G','30cm','30g','30mL','330mL','34g','35mm','360mm','37mm','38cm','38mm','3cc','3mL','3mm','4,24cm','4,4cm','4.01g','4.35g','4.54g','4.5cm','4.5m','4.66g','400g','400ml','402G','40cm','410cm','43L','43cm','44cm','450g','45cm','470mm','4L','5.17g','5.5cm','5.7g','500g','500mL','50L','50cm','50g','50mL','50ml','50mm','51cm','54cm','5540cm','55m','58L','5L','5cc','5cm','5g','5m','5mL','600g','60cm','60g','60mL','615cm','63L','65mm','6cm','7.5cm','7.5g','75cm','75g','76cm','7cm','7mm','70%','804L','80cm','80g','82L','83%','85cm','8g','9.0g','9.1m','9.5cm','90cm','90g','91cm','95g','9g','M601G','S748G']
org = ['국산','남아프리카공화국','뉴질랜드','러시아','미국','미얀마','베트남','우즈베키스탄','이집트','인도네시아','중국','콜롬비아','콩고','키르키즈스탄','태국','파키스탄','페루','한국']
herb = ['가자','갈근','갈화','감송향','감수','감초','감초초','강활','강황','건강','건강초','건율','검인','견우자','결명자','계지','계혈등','고량강','고본','곡기생','골담초근','과체','곽향','관중','괄루근','괄루인','광곽향','괴각','괴화','구기자','구맥','구자','구척','국화','귀판교','금앵자','금은화','금전초','길경','내복자','내복자초','노근','노로통','녹각교가루','녹각교질','녹두','녹용','녹용상대','누로','단삼','당귀','당삼','대두황권','대복피','대자석','대청엽','대추','대풍자','대황','대황주증','도인','독활','동과자','두시','두충','두충염자','등심초','마인','마치현','마황','마황탕포','만형자','망초','맥문동','맥아','맥아초','맹충','면실자','모과','모근','모려','모려가루','모려단','목과','목단피','목통','목향','밀몽화','바이오닷','박하','반묘','반지련','반하','생강백반제','방기','방풍','백강잠','백과','백두구','백두옹','백렴','백반','백부근','백부자','백자인','백지','백출','백출미감침','백출초','백편두','백편두초','백합','별갑','별갑교','보골지염자','복령','부자','부평','비파엽','빈랑자','사간','사과락','사삼','사상자','사원자','사인','사인초','사향','산내','산사','산사초','산수유','산약','산조인','산조인초','산초','삼릉','삼백초','상기생','상백피','상산','상심자','상표초','생강','석결명','석결명가루','석고','석고가루','석류','석유황','석창포','선복화','선퇴','세신','소목','속단','쇄양','수오등','숙지황','스윗팜','승마','시라자','시호','식방풍','신곡초','신곡홍육','신근초','신이','아교','아마인','아선약','아출','애엽','어성초','여정실','여정실주증','여지핵','연교','연자육','영릉향','영양각','오가피','오공','오매','오미자','오배자','오수유','오수유감초자','오수유염자','오약','옥촉서예','올본','용골가루','용골단','용규','용담','용안육','우담남성','우방자','우슬','우슬주자','우황','욱리인','운모','울금','원지','원지감초자','원지밀자','위령선','유향','육계','육두구','육종용주증','은시호','의이인','이풀잎','익모초','익지','인동','인삼','일당귀','자근','자석영','자소엽','자소자','자연동','자완','자충','자화지정','작약','작약주자','작약초','잠사','저령','적석지','전충','전호','절패모','접골목','정공등','정력자','정향','제니','제조','조구등','죽력','죽여','지각','지골피','지구자','지모','지모염자','지부자','지실','지실초','지유','지유초탄','지황','진교','진피','진피초','질려자','차전자','창이자','창출','창출미감침','창출초','천궁','천남성','천마','천문동','천오','천오제','청피','초두구','초오','초오제','충위자','치자','치자초','택란','택사','토복령','토사자','토사자주증','파극천주자','판람근','패장','편축','포공영','포부자','포황','피마자','필발','하고초','하수오','하수오주증','하수오흑두즙돈','하엽','한련초','해동피','해방풍','해표초','해표초가루','행인','향부자','향부자초','향부자초자','현삼','현초','현호색','형개','호로파','호박','호장근','홍화','홍화자','화피','활석','황금','황금초','황기','황기밀자','황련','황련주자','황백','황백염자','황백주자','황정','황정주증','후박','후박강자','후추']
model = ['01M3', '01M4', '01M7', '101A', '101B', '101C', '101D', '101E', '101F', '101G', '101H', '101J', '101K', '101R', '101S', '101T', '101U', '150W', '2002A', '2002B', '2002C', '2002D', '2003A', '2003B', '2004A', '2004B', '2004C', '2004D', '2004E', '2004F', '2004G', '2004H', '201A', '201B', '201C', '201D', '201E', '201F', '201G', '201H', '201J', '201K', '201R', '201S', '201T', '201U', '300L', '301-A', '301A', '301B', '301C', '301D', '301E', '301F', '301G', '301H', '301J', '301K', '301M', '302A', '302B', '302C', '302D', '302E', '302F', '302G', '302H', '302J', '302K', '302M', '370S', '3S', '50S', '601A', '601B', '7202H', '900C', '900T', '9R', 'A1', 'A10', 'A100', 'A11', 'A12', 'A15', 'A18', 'A18-6', 'A18/1', 'A18/21', 'A18/5', 'A20', 'A20-2', 'A250', 'A283', 'A300', 'A4', 'A41', 'A52', 'A58', 'A58-2', 'A58-3', 'A58-7', 'A58_6', 'A59', 'A59-8', 'A59_2', 'A72', 'A76', 'A76-5', 'A795', 'A80', 'A80-1', 'A81', 'A82', 'A82-1', 'A83', 'A83-1', 'A84-1', 'A85-1', 'A86-1', 'A87-1', 'A88', 'A880', 'A881', 'A882', 'A883', 'A89', 'AFIAS-1', 'AFIAS-6', 'AL100', 'AL102', 'AL104', 'AL105', 'AL106', 'AL107', 'AL108', 'AL109', 'AL110', 'AL113', 'AL116', 'AL117', 'AL120', 'AL121', 'AL125', 'AL126', 'AL127', 'AL128', 'AL129', 'AL138', 'AL139', 'AL147', 'AL155', 'AL160', 'AL161', 'AL163', 'AL165', 'AT-1100', 'AT-562', 'A급', 'A형', 'B01', 'B09', 'B11', 'B13', 'B17', 'B19', 'B22', 'B223', 'B235', 'B30', 'B32', 'B35', 'B55', 'B59', 'B60', 'BC300', 'BC380', 'BC720', 'BM-420', 'BM-760', 'BMS-1000', 'BP170', 'BP170B', 'BP210', 'BP500', 'BP850', 'BPBIO210', 'BPBIO220', 'BPBIO250', 'BT-3000', 'B형', 'C12', 'C13', 'C14', 'C19', 'C22', 'C25', 'C30', 'C38', 'CK-110', 'CK-112P', 'CK-113P', 'CK-142', 'CK-143', 'CK-152', 'CK-153', 'CK-501', 'CK-502', 'CK-503', 'CK-504', 'CK-505', 'CK-506', 'CK-507', 'CK-601PF', 'CK-603PW', 'CK-648CPF', 'CK-648DPF', 'CK-648PF', 'CK-649CWT', 'CK-900', 'CK-901', 'CK-902', 'CK-906', 'CK-907', 'CK-907A', 'CK-908', 'CK-909', 'CK-910', 'CK-911', 'CK-939', 'CK-939A', 'CK-A603T', 'CK-A604T', 'CK-A605T', 'CK-E301', 'CK-E301A', 'CK-F606PF', 'CK-M601GPF', 'CK-S601CPF', 'CK-S601PF', 'CK-S606PF', 'CK-S607P', 'CK-S621PF', 'CK-S748GPF', 'CK-S748PF', 'CK-SS601CPF', 'CK-SS601PF', 'CK-SS747CPF', 'CK-SS747PF', 'CK903', 'CRYO-JET', 'CWM-402', 'CY1016', 'CY1017', 'CY1040', 'CY1060', 'CY1080', 'CY1089', 'CY1090', 'CY1092', 'CY1140', 'CY1160', 'CY1180', 'CY3010', 'CY3011', 'CY3020', 'CY3030', 'CY3033', 'CY3040', 'CY3050', 'CY3055', 'CY3077', 'CY3099', 'CY3150', 'CY3160', 'CY3180', 'CY3190', 'CY3210', 'CY3220', 'CY3240', 'CY3321', 'CY3341', 'Cettum-S', 'CettumⅡ', 'C형', 'D17', 'D4', 'DC-40', 'DC-60', 'DC-N3', 'DCS104T', 'DMP-1000PLUS', 'DMP-LIFE', 'DMP-LIFEPLUS', 'E-400', 'E-440', 'E-CUBE', 'E100', 'E102', 'E104', 'E12', 'E200', 'E202', 'E203', 'E300', 'ELS-100', 'ES-160', 'ESWT-1000', 'F13', 'FMD-8000', 'FN_1', 'G-500', 'G13', 'G20', 'G21', 'G230', 'G240', 'G241', 'G30', 'G40', 'GHB-2200', 'GP-102H', 'GP-12PD', 'GP-304N', 'GP-512D', 'GP-707SW', 'GP-804L', 'GP-900M', 'H20-1', 'H20-3', 'H20-4', 'H200', 'H20B', 'H20N', 'H3', 'H400', 'H500', 'HA-306', 'HAS-1300', 'HBP-1320', 'HBP-9030', 'HCR-1602', 'HDT1000', 'HEALTRON', 'HEALTRON-S', 'HEM-7141', 'HM-801C', 'HM-801R', 'HS-401F3', 'HS-401F8', 'HS-402G2', 'HS-502', 'HS-OP10', 'HS-OT10', 'HSP-601', 'HTS-24F', 'I-2000', 'ICHROMA2', 'IN-2300', 'INBIO-500', 'INSIGHT-I', 'INW-500NS', 'IR-VITA', 'IR300', 'IR300A', 'IS-400', 'IWS-7000', 'J213', 'J50', 'K09', 'K15', 'K15-1', 'K16', 'K17', 'K18', 'K20', 'K21', 'K217', 'K218', 'K22-2', 'K22-3', 'K23', 'K24', 'K25', 'K26', 'K27', 'K32', 'K41', 'K55', 'KF-AD', 'KF94', 'KH14190', 'KH15200', 'KH16210', 'KL300', 'KRS-0505B', 'KRS-0505C', 'KRS-12PA', 'KRS-12PD', 'KRS-200D', 'KRS-202D', 'KRS-202HG', 'KRS-203', 'KRS-204', 'KRS-205', 'KRS-24PA', 'KRS-24PD', 'KRS-8PD', 'KRS-985', 'KRS-989', 'KRS-A1', 'KS-100', 'KS-105', 'KS-135', 'KS-220', 'KS-450C', 'Kinetrac-9900', 'Knx-7000', 'LS-330M', 'M10', 'M18', 'M2-A', 'M2-B', 'M2-C', 'M2-H', 'M20', 'M21', 'M22', 'M30', 'M31', 'M32', 'M33', 'M33-1', 'M34', 'M34-1', 'MEK-6510K', 'MK-10', 'MK-25S', 'MK-50S', 'MPS-1', 'MS-3000', 'MS-660M', 'MS-880', 'MT-7000R', 'MX-90', 'N100', 'N102', 'NB-810B', 'NET-1100', 'NET-280', 'NET-600C', 'NP-30', 'NP-70', 'NP-77', 'NX500i', 'Neuro-MSL', 'Omelon', 'PL330', 'PL550', 'PM-606', 'PM-808A', 'PT10', 'PaR38', 'P형', 'Q환', 'R3035', 'R3060', 'R60200', 'R6060', 'R95', 'RS-1000', 'RS-2100', 'RS-3000', 'RS-5000', 'RX110', 'Raphael', 'S-20', 'S-SLIM', 'S10', 'SA-3000P', 'SRT-1100', 'ST-10A', 'ST-30A', 'STBP-500', 'STD-1000', 'STI-500', 'STM-1000', 'STM-900', 'STN-110', 'STN-330', 'STS-1000', 'STT-570', 'T-1000', 'T-193', 'T-194', 'T-331BL', 'T-SONIC', 'U자형', 'VB84', 'VE315', 'VERI-Q', 'VYPER2.0', 'W19007', 'W2009A', 'W2009B', 'WA14533', 'WA14555', 'X-CUBE', 'dw-990', 'is-300', 'pmi-3000', '다이얼W']
mdwords = list(set(op1 + op2 + herb+ model + bnm+ org))
twitter.add_dictionary(mdwords, 'Noun')
# twitter.add_dictionary(op1, 'Noun')

## morphs

In [16]:
# twitter 돌리고 corpus에 넣을 코드 짜기
# medistream_morphs : twitter으로 토큰들을 쪼개 리스트에 넣기(상품명 하나가 하나의 리스트에 들어간 중첩리스트)
medistream_morphs = []
akom_morphs = []
anjin_morphs = []
hane_morphs = []
hanpure_morphs = []
human_hub_morphs = []
km_mall_morphs = []
daejoong_morphs = []
for i in medistream_rex_list:
    medistream_morphs.append(twitter.morphs(i))

for i in akom_rex_list:
    akom_morphs.append(twitter.morphs(i))

for i in anjin_rex_list:
    anjin_morphs.append(twitter.morphs(i))

for i in hane_rex_list:
    hane_morphs.append(twitter.morphs(i))

for i in hanpure_name_list:
    hanpure_morphs.append(twitter.morphs(i))

for i in human_hub_name_list:
    human_hub_morphs.append(twitter.morphs(i))

for i in km_mall_rex_list:
    km_mall_morphs.append(twitter.morphs(i))

for i in daejoong_rex_list:
    daejoong_morphs.append(twitter.morphs(i))

In [17]:
# 상품명 하나씩 공백 추가해서 리스트에 넣기(tfidf에서 split으로 처리하기 때문)
medistream_corpus = []
akom_corpus = []
anjin_corpus = []
hane_corpus = []
hanpure_corpus = []
human_hub_corpus = []
km_mall_corpus = []
daejoong_corpus = []

for i in range(len(medistream_morphs)):
    medistream_corpus.append((' ').join(medistream_morphs[i]))
for i in range(len(akom_morphs)):
    akom_corpus.append((' ').join(akom_morphs[i]))
for i in range(len(anjin_morphs)):
    anjin_corpus.append((' ').join(anjin_morphs[i]))
for i in range(len(hane_morphs)):
    hane_corpus.append((' ').join(hane_morphs[i]))
for i in range(len(hanpure_morphs)):
    hanpure_corpus.append((' ').join(hanpure_morphs[i]))
for i in range(len(human_hub_morphs)):
    human_hub_corpus.append((' ').join(human_hub_morphs[i]))
for i in range(len(km_mall_morphs)):
    km_mall_corpus.append((' ').join(km_mall_morphs[i]))
for i in range(len(daejoong_morphs)):
    daejoong_corpus.append((' ').join(daejoong_morphs[i]))

medistream_akom_corpus = medistream_corpus + akom_corpus
medistream_anjin_corpus = medistream_corpus + anjin_corpus
medistream_hane_corpus = medistream_corpus + hane_corpus
medistream_hanpure_corpus = medistream_corpus + hanpure_corpus
medistream_human_hub_corpus = medistream_corpus + human_hub_corpus
medistream_km_mall_corpus = medistream_corpus + km_mall_corpus
medistream_daejoong_corpus = medistream_corpus + daejoong_corpus

# TF-IDF

In [18]:
# TfidfVectorizer
medistream_akom_tfidf_vector = TfidfVectorizer().fit(medistream_akom_corpus)
medistream_anjin_tfidf_vector = TfidfVectorizer().fit(medistream_anjin_corpus)
medistream_hane_tfidf_vector = TfidfVectorizer().fit(medistream_hane_corpus)
medistream_hanpure_tfidf_vector = TfidfVectorizer().fit(medistream_hanpure_corpus)
medistream_human_hub_tfidf_vector = TfidfVectorizer().fit(medistream_human_hub_corpus)
medistream_km_mall_tfidf_vector = TfidfVectorizer().fit(medistream_km_mall_corpus)
medistream_daejoong_tfidf_vector = TfidfVectorizer().fit(medistream_daejoong_corpus)

# 만들어진 tfidf행렬을 각각의 변수에 바인딩
medistream_akom_tfidf = medistream_akom_tfidf_vector.transform(medistream_akom_corpus).toarray() 
medistream_anjin_tfidf = medistream_anjin_tfidf_vector.transform(medistream_anjin_corpus).toarray() 
medistream_hane_tfidf = medistream_hane_tfidf_vector.transform(medistream_hane_corpus).toarray() 
medistream_hanpure_tfidf = medistream_hanpure_tfidf_vector.transform(medistream_hanpure_corpus).toarray() 
medistream_human_hub_tfidf = medistream_human_hub_tfidf_vector.transform(medistream_human_hub_corpus).toarray() 
medistream_km_mall_tfidf = medistream_km_mall_tfidf_vector.transform(medistream_km_mall_corpus).toarray() 
medistream_daejoong_tfidf = medistream_daejoong_tfidf_vector.transform(medistream_daejoong_corpus).toarray() 

# 코사인 유사도

In [19]:
# 코사인 유사도 함수
def cos_sim(A, B):
        return dot(A, B)/(norm(A)*norm(B))

# 인덱스 찾기

In [20]:
medistream_id_dic = {}
medistream_price_dic = {}
medistream_rex_dic = {}

medistream_akom_tfidf_for_sim = {}
medistream_anjin_tfidf_for_sim = {}
medistream_hane_tfidf_for_sim = {}
medistream_hanpure_tfidf_for_sim = {}
medistream_human_hub_tfidf_for_sim = {}
medistream_km_mall_tfidf_for_sim = {}
medistream_daejoong_tfidf_for_sim = {}

for i in range(len(medistream_name_list)):
    medistream_id_dic[medistream_name_list[i].strip()] = medistream_id_list[i]
    medistream_price_dic[medistream_name_list[i].strip()] = medistream_price_list[i]
    medistream_rex_dic[medistream_name_list[i].strip()] = medistream_rex_list[i]

    medistream_akom_tfidf_for_sim[medistream_name_list[i].strip()] = medistream_akom_tfidf[i]
    medistream_anjin_tfidf_for_sim[medistream_name_list[i].strip()] = medistream_anjin_tfidf[i]
    medistream_hane_tfidf_for_sim[medistream_name_list[i].strip()] = medistream_hane_tfidf[i]
    medistream_hanpure_tfidf_for_sim[medistream_name_list[i].strip()] = medistream_hanpure_tfidf[i]
    medistream_human_hub_tfidf_for_sim[medistream_name_list[i].strip()] = medistream_human_hub_tfidf[i]
    medistream_km_mall_tfidf_for_sim[medistream_name_list[i].strip()] = medistream_km_mall_tfidf[i]
    medistream_daejoong_tfidf_for_sim[medistream_name_list[i].strip()] = medistream_daejoong_tfidf[i]

# 아콤몰

In [21]:
# 아콤몰 // 메디스트림 리스트는 인덱스 채우기용
akom_for_marekt_result = medistream_name_list + akom_market_name_list
akom_for_name_result = medistream_name_list + akom_name_list 
akom_for_price_result = medistream_name_list + akom_price_list
akom_for_id_result = medistream_name_list + akom_id_list
akom_for_url_result = medistream_name_list + akom_url_list
akom_for_rex_result = medistream_name_list + akom_rex_list

# 아콤몰 인덱스 찾기 위한 딕셔너리
akom_index_search = {}

for i in range(len(akom_rex_list)):
    akom_index_search[akom_rex_list[i].strip()] = i

In [22]:
def akom_func(medistream_name_input):

    medistream_rex_for_comparison = medistream_rex_dic[medistream_name_input.strip()]

    akom1 = []
    akom2 = []

    akom3 = []
    akom4 = []
    akom5 = []
    akom6 = []
    akom7 = []




    len1 = len(medistream_name_list)
    len2 = len(akom_name_list)
    # 메디스트림 상품을 제외하고 시작
    for i in range(len1, len1+len2):

        akom_result = cos_sim(medistream_akom_tfidf_for_sim[medistream_name_input.strip()], medistream_akom_tfidf[i])
        if  akom_result >= 0.1:
            akom1.append(akom_for_rex_result[i])
            akom2.append(akom_result)


        akom_df = pd.DataFrame({'정규표현식_상품명': akom1,
                            '유사도': akom2})
        
        # 쇼핑몰에서 정답 topk개 출력
        akom_sort_results = akom_df.sort_values(by=['유사도'],ascending=False)

        # topk내에서 2차 매칭
        akom_new_topk = {}
        akom_val_list = []
        akom_topks = list(akom_sort_results['정규표현식_상품명'].head(10))

        if len(akom_topks) > 0:
            for i in range(len(akom_topks)):
                akom_new = akom_topks[i]
                akom_token = twitter.morphs(akom_new)
                akom_cnt = 0
                akom_same = []
                for i in range(len(akom_token)):
                    akom_comp = (akom_token[i], medistream_rex_for_comparison.find(akom_token[i]))
                    if akom_comp[1] != -1:
                        akom_cnt += 1
                        akom_same.append(akom_token[i])
                # 타사 상품명:일치 개수
                akom_new_topk[akom_new] = akom_cnt

        # akom_new_topk에서 akom_cnt순으로 top 5뽑기 
        akom_sort_new = dict(sorted(akom_new_topk.items(), key = lambda item: item[1], reverse = True))
        akom_new_list = list(akom_sort_new.keys())[:5]

    akom_last_name_list = []
    for i in akom_new_list:
        akom_last_name_list.append(i.strip())


    for i in akom_last_name_list:
        akom3.append(akom_market_name_list[akom_index_search[i]])
        akom4.append(akom_name_list[akom_index_search[i]])
        akom5.append(akom_price_list[akom_index_search[i]])
        akom6.append(akom_id_list[akom_index_search[i]])
        akom7.append(akom_url_list[akom_index_search[i]])





    akom_last_df = pd.DataFrame({'쇼핑몰': akom3,
                        '상품명': akom4,
                        '가격': akom5,
                        'id': akom6,
                        'url': akom7})

    return akom_last_df

# 안진메디팜

In [23]:
# 안진메디팜 // 메디스트림 리스트는 인덱스 채우기용
anjin_for_marekt_result = medistream_name_list + anjin_market_name_list
anjin_for_name_result = medistream_name_list + anjin_name_list 
anjin_for_price_result = medistream_name_list + anjin_price_list
anjin_for_id_result = medistream_name_list + anjin_id_list
anjin_for_url_result = medistream_name_list + anjin_url_list
anjin_for_rex_result = medistream_name_list + anjin_rex_list

# 안진메디팜 인덱스 찾기 위한 딕셔너리
anjin_index_search = {}

for i in range(len(anjin_rex_list)):
    anjin_index_search[anjin_rex_list[i].strip()] = i

In [24]:
def anjin_func(medistream_name_input):
    medistream_rex_for_comparison = medistream_rex_dic[medistream_name_input.strip()]

    anjin1 = []
    anjin2 = []

    anjin3 = []
    anjin4 = []
    anjin5 = []
    anjin6 = []
    anjin7 = []



    len1 = len(medistream_name_list)
    len3 = len(anjin_name_list)
    # 메디스트림 상품을 제외하고 시작
    for i in range(len1, len1+len3):

        anjin_result = cos_sim(medistream_anjin_tfidf_for_sim[medistream_name_input.strip()], medistream_anjin_tfidf[i])
        if  anjin_result >= 0.1:
            anjin1.append(anjin_for_rex_result[i])
            anjin2.append(anjin_result)


        anjin_df = pd.DataFrame({'정규표현식_상품명': anjin1,
                            '유사도': anjin2})
        
        # 쇼핑몰에서 정답 topk개 출력
        anjin_sort_results = anjin_df.sort_values(by=['유사도'],ascending=False)

        # topk내에서 2차 매칭
        anjin_new_topk = {}
        anjin_val_list = []
        anjin_topks = list(anjin_sort_results['정규표현식_상품명'].head(10))

        if len(anjin_topks) > 0:
            for i in range(len(anjin_topks)):
                anjin_new = anjin_topks[i]
                anjin_token = twitter.morphs(anjin_new)
                anjin_cnt = 0
                anjin_same = []
                for i in range(len(anjin_token)):
                    anjin_comp = (anjin_token[i], medistream_rex_for_comparison.find(anjin_token[i]))
                    if anjin_comp[1] != -1:
                        anjin_cnt += 1
                        anjin_same.append(anjin_token[i])
                # 타사 상품명:일치 개수
                anjin_new_topk[anjin_new] = anjin_cnt

        # anjin_new_topk에서 anjin_cnt순으로 top 5뽑기 
        anjin_sort_new = dict(sorted(anjin_new_topk.items(), key = lambda item: item[1], reverse = True))
        anjin_new_list = list(anjin_sort_new.keys())[:5]

    anjin_last_name_list = []
    for i in anjin_new_list:
        anjin_last_name_list.append(i.strip())


    for i in anjin_last_name_list:
        anjin3.append(anjin_market_name_list[anjin_index_search[i]])
        anjin4.append(anjin_name_list[anjin_index_search[i]])
        anjin5.append(anjin_price_list[anjin_index_search[i]])
        anjin6.append(anjin_id_list[anjin_index_search[i]])
        anjin7.append(anjin_url_list[anjin_index_search[i]])





    anjin_last_df = pd.DataFrame({'쇼핑몰': anjin3,
                        '상품명': anjin4,
                        '가격': anjin5,
                        'id': anjin6,
                        'url': anjin7})

    return anjin_last_df

# 한의몰

In [25]:
# 한의몰 // 메디스트림 리스트는 인덱스 채우기용
hane_for_marekt_result = medistream_name_list + hane_market_name_list
hane_for_name_result = medistream_name_list + hane_name_list 
hane_for_price_result = medistream_name_list + hane_price_list
hane_for_id_result = medistream_name_list + hane_id_list
hane_for_url_result = medistream_name_list + hane_url_list
hane_for_rex_result = medistream_name_list + hane_rex_list

# 한의몰 인덱스 찾기 위한 딕셔너리
hane_index_search = {}

for i in range(len(hane_rex_list)):
    hane_index_search[hane_rex_list[i].strip()] = i

In [26]:
def hane_func(medistream_name_input):

    medistream_rex_for_comparison = medistream_rex_dic[medistream_name_input.strip()]

    hane1 = []
    hane2 = []

    hane3 = []
    hane4 = []
    hane5 = []
    hane6 = []
    hane7 = []



    len1 = len(medistream_name_list)
    len4 = len(hane_name_list)
    # 메디스트림 상품을 제외하고 시작
    for i in range(len1, len1+len4):

        hane_result = cos_sim(medistream_hane_tfidf_for_sim[medistream_name_input.strip()], medistream_hane_tfidf[i])
        if  hane_result >= 0.1:
            hane1.append(hane_for_rex_result[i])
            hane2.append(hane_result)


        hane_df = pd.DataFrame({'정규표현식_상품명': hane1,
                            '유사도': hane2})
        
        # 쇼핑몰에서 정답 topk개 출력
        hane_sort_results = hane_df.sort_values(by=['유사도'],ascending=False)

        # topk내에서 2차 매칭
        hane_new_topk = {}
        hane_val_list = []
        hane_topks = list(hane_sort_results['정규표현식_상품명'].head(10))

        if len(hane_topks) > 0:
            for i in range(len(hane_topks)):
                hane_new = hane_topks[i]
                hane_token = twitter.morphs(hane_new)
                hane_cnt = 0
                hane_same = []
                for i in range(len(hane_token)):
                    hane_comp = (hane_token[i], medistream_rex_for_comparison.find(hane_token[i]))
                    if hane_comp[1] != -1:
                        hane_cnt += 1
                        hane_same.append(hane_token[i])
                # 타사 상품명:일치 개수
                hane_new_topk[hane_new] = hane_cnt

        # hane_new_topk에서 hane_cnt순으로 top 5뽑기 
        hane_sort_new = dict(sorted(hane_new_topk.items(), key = lambda item: item[1], reverse = True))
        hane_new_list = list(hane_sort_new.keys())[:5]

    hane_last_name_list = []
    for i in hane_new_list:
        hane_last_name_list.append(i.strip())


    for i in hane_last_name_list:
        hane3.append(hane_market_name_list[hane_index_search[i]])
        hane4.append(hane_name_list[hane_index_search[i]])
        hane5.append(hane_price_list[hane_index_search[i]])
        hane6.append(hane_id_list[hane_index_search[i]])
        hane7.append(hane_url_list[hane_index_search[i]])





    hane_last_df = pd.DataFrame({'쇼핑몰': hane3,
                        '상품명': hane4,
                        '가격': hane5,
                        'id': hane6,
                        'url': hane7})

    return hane_last_df

# 한퓨어몰

In [27]:
# 한퓨어몰 // 메디스트림 리스트는 인덱스 채우기용
hanpure_for_marekt_result = medistream_name_list + hanpure_market_name_list
hanpure_for_name_result = medistream_name_list + hanpure_name_list 
hanpure_for_price_result = medistream_name_list + hanpure_price_list
hanpure_for_id_result = medistream_name_list + hanpure_id_list
hanpure_for_url_result = medistream_name_list + hanpure_url_list
hanpure_for_rex_result = medistream_name_list + hanpure_name_list

# 한퓨어몰 인덱스 찾기 위한 딕셔너리
hanpure_index_search = {}

for i in range(len(hanpure_name_list)):
    hanpure_index_search[hanpure_name_list[i].strip()] = i

In [28]:
def hanpure_func(medistream_name_input):
    
    medistream_rex_for_comparison = medistream_rex_dic[medistream_name_input.strip()]

    hanpure1 = []
    hanpure2 = []

    hanpure3 = []
    hanpure4 = []
    hanpure5 = []
    hanpure6 = []
    hanpure7 = []



    len1 = len(medistream_name_list)
    len5 = len(hanpure_name_list)
    # 메디스트림 상품을 제외하고 시작
    for i in range(len1, len1+len5):

        hanpure_result = cos_sim(medistream_hanpure_tfidf_for_sim[medistream_name_input.strip()], medistream_hanpure_tfidf[i])
        if  hanpure_result >= 0.1:
            hanpure1.append(hanpure_for_rex_result[i])
            hanpure2.append(hanpure_result)


        hanpure_df = pd.DataFrame({'정규표현식_상품명': hanpure1,
                            '유사도': hanpure2})
        
        # 쇼핑몰에서 정답 topk개 출력
        hanpure_sort_results = hanpure_df.sort_values(by=['유사도'],ascending=False)

        # topk내에서 2차 매칭
        hanpure_new_topk = {}
        hanpure_val_list = []
        hanpure_topks = list(hanpure_sort_results['정규표현식_상품명'].head(10))

        if len(hanpure_topks) > 0:
            for i in range(len(hanpure_topks)):
                hanpure_new = hanpure_topks[i]
                hanpure_token = twitter.morphs(hanpure_new)
                hanpure_cnt = 0
                hanpure_same = []
                for i in range(len(hanpure_token)):
                    hanpure_comp = (hanpure_token[i], medistream_rex_for_comparison.find(hanpure_token[i]))
                    if hanpure_comp[1] != -1:
                        hanpure_cnt += 1
                        hanpure_same.append(hanpure_token[i])
                # 타사 상품명:일치 개수
                hanpure_new_topk[hanpure_new] = hanpure_cnt

        # hanpure_new_topk에서 hanpure_cnt순으로 top 5뽑기 
        hanpure_sort_new = dict(sorted(hanpure_new_topk.items(), key = lambda item: item[1], reverse = True))
        hanpure_new_list = list(hanpure_sort_new.keys())[:5]

    hanpure_last_name_list = []
    for i in hanpure_new_list:
        hanpure_last_name_list.append(i.strip())


    for i in hanpure_last_name_list:
        hanpure3.append(hanpure_market_name_list[hanpure_index_search[i]])
        hanpure4.append(hanpure_name_list[hanpure_index_search[i]])
        hanpure5.append(hanpure_price_list[hanpure_index_search[i]])
        hanpure6.append(hanpure_id_list[hanpure_index_search[i]])
        hanpure7.append(hanpure_url_list[hanpure_index_search[i]])





    hanpure_last_df = pd.DataFrame({'쇼핑몰': hanpure3,
                        '상품명': hanpure4,
                        '가격': hanpure5,
                        'id': hanpure6,
                        'url': hanpure7})

    return hanpure_last_df

# 휴먼허브

In [29]:
# 휴먼허브 // 메디스트림 리스트는 인덱스 채우기용
human_hub_for_marekt_result = medistream_name_list + human_hub_market_name_list
human_hub_for_name_result = medistream_name_list + human_hub_name_list 
human_hub_for_price_result = medistream_name_list + human_hub_price_list
human_hub_for_id_result = medistream_name_list + human_hub_id_list
human_hub_for_url_result = medistream_name_list + human_hub_url_list
human_hub_for_rex_result = medistream_name_list + human_hub_name_list

# 휴먼허브 인덱스 찾기 위한 딕셔너리
human_hub_index_search = {}

for i in range(len(human_hub_name_list)):
    human_hub_index_search[human_hub_name_list[i].strip()] = i

In [30]:
def human_hub_func(medistream_name_input):

    medistream_rex_for_comparison = medistream_rex_dic[medistream_name_input.strip()]

    human_hub1 = []
    human_hub2 = []

    human_hub3 = []
    human_hub4 = []
    human_hub5 = []
    human_hub6 = []
    human_hub7 = []



    len1 = len(medistream_name_list)
    len6 = len(human_hub_name_list)
    # 메디스트림 상품을 제외하고 시작
    for i in range(len1, len1+len6):

        human_hub_result = cos_sim(medistream_human_hub_tfidf_for_sim[medistream_name_input.strip()], medistream_human_hub_tfidf[i])
        if  human_hub_result >= 0.1:
            human_hub1.append(human_hub_for_rex_result[i])
            human_hub2.append(human_hub_result)


        human_hub_df = pd.DataFrame({'정규표현식_상품명': human_hub1,
                            '유사도': human_hub2})
        
        # 쇼핑몰에서 정답 topk개 출력
        human_hub_sort_results = human_hub_df.sort_values(by=['유사도'],ascending=False)

        # topk내에서 2차 매칭
        human_hub_new_topk = {}
        human_hub_val_list = []
        human_hub_topks = list(human_hub_sort_results['정규표현식_상품명'].head(10))

        if len(human_hub_topks) > 0:
            for i in range(len(human_hub_topks)):
                human_hub_new = human_hub_topks[i]
                human_hub_token = twitter.morphs(human_hub_new)
                human_hub_cnt = 0
                human_hub_same = []
                for i in range(len(human_hub_token)):
                    human_hub_comp = (human_hub_token[i], medistream_rex_for_comparison.find(human_hub_token[i]))
                    if human_hub_comp[1] != -1:
                        human_hub_cnt += 1
                        human_hub_same.append(human_hub_token[i])
                # 타사 상품명:일치 개수
                human_hub_new_topk[human_hub_new] = human_hub_cnt

        # human_hub_new_topk에서 human_hub_cnt순으로 top 5뽑기 
        human_hub_sort_new = dict(sorted(human_hub_new_topk.items(), key = lambda item: item[1], reverse = True))
        human_hub_new_list = list(human_hub_sort_new.keys())[:5]

    human_hub_last_name_list = []
    for i in human_hub_new_list:
        human_hub_last_name_list.append(i.strip())


    for i in human_hub_last_name_list:
        human_hub3.append(human_hub_market_name_list[human_hub_index_search[i]])
        human_hub4.append(human_hub_name_list[human_hub_index_search[i]])
        human_hub5.append(human_hub_price_list[human_hub_index_search[i]])
        human_hub6.append(human_hub_id_list[human_hub_index_search[i]])
        human_hub7.append(human_hub_url_list[human_hub_index_search[i]])





    human_hub_last_df = pd.DataFrame({'쇼핑몰': human_hub3,
                        '상품명': human_hub4,
                        '가격': human_hub5,
                        'id': human_hub6,
                        'url': human_hub7})

    return human_hub_last_df

# 케이엠몰

In [31]:
# 케이엠몰 // 메디스트림 리스트는 인덱스 채우기용
km_mall_for_marekt_result = medistream_name_list + km_mall_market_name_list
km_mall_for_name_result = medistream_name_list + km_mall_name_list 
km_mall_for_price_result = medistream_name_list + km_mall_price_list
km_mall_for_id_result = medistream_name_list + km_mall_id_list
km_mall_for_url_result = medistream_name_list + km_mall_url_list
km_mall_for_rex_result = medistream_name_list + km_mall_rex_list

# 케이엠몰 인덱스 찾기 위한 딕셔너리
km_mall_index_search = {}

for i in range(len(km_mall_rex_list)):
    km_mall_index_search[km_mall_rex_list[i].strip()] = i

In [32]:
def km_mall_func(medistream_name_input):

    medistream_rex_for_comparison = medistream_rex_dic[medistream_name_input.strip()]

    km_mall1 = []
    km_mall2 = []

    km_mall3 = []
    km_mall4 = []
    km_mall5 = []
    km_mall6 = []
    km_mall7 = []




    len1 = len(medistream_name_list)
    len7 = len(km_mall_name_list)
    # 메디스트림 상품을 제외하고 시작
    for i in range(len1, len1+len7):

        km_mall_result = cos_sim(medistream_km_mall_tfidf_for_sim[medistream_name_input.strip()], medistream_km_mall_tfidf[i])
        if  km_mall_result >= 0.1:
            km_mall1.append(km_mall_for_rex_result[i])
            km_mall2.append(km_mall_result)


        km_mall_df = pd.DataFrame({'정규표현식_상품명': km_mall1,
                            '유사도': km_mall2})
        
        # 쇼핑몰에서 정답 topk개 출력
        km_mall_sort_results = km_mall_df.sort_values(by=['유사도'],ascending=False)

        # topk내에서 2차 매칭
        km_mall_new_topk = {}
        km_mall_val_list = []
        km_mall_topks = list(km_mall_sort_results['정규표현식_상품명'].head(10))

        if len(km_mall_topks) > 0:
            for i in range(len(km_mall_topks)):
                km_mall_new = km_mall_topks[i]
                km_mall_token = twitter.morphs(km_mall_new)
                km_mall_cnt = 0
                km_mall_same = []
                for i in range(len(km_mall_token)):
                    km_mall_comp = (km_mall_token[i], medistream_rex_for_comparison.find(km_mall_token[i]))
                    if km_mall_comp[1] != -1:
                        km_mall_cnt += 1
                        km_mall_same.append(km_mall_token[i])
                # 타사 상품명:일치 개수
                km_mall_new_topk[km_mall_new] = km_mall_cnt

        # km_mall_new_topk에서 km_mall_cnt순으로 top 5뽑기 
        km_mall_sort_new = dict(sorted(km_mall_new_topk.items(), key = lambda item: item[1], reverse = True))
        km_mall_new_list = list(km_mall_sort_new.keys())[:5]

    km_mall_last_name_list = []
    for i in km_mall_new_list:
        km_mall_last_name_list.append(i.strip())


    for i in km_mall_last_name_list:
        km_mall3.append(km_mall_market_name_list[km_mall_index_search[i]])
        km_mall4.append(km_mall_name_list[km_mall_index_search[i]])
        km_mall5.append(km_mall_price_list[km_mall_index_search[i]])
        km_mall6.append(km_mall_id_list[km_mall_index_search[i]])
        km_mall7.append(km_mall_url_list[km_mall_index_search[i]])





    km_mall_last_df = pd.DataFrame({'쇼핑몰': km_mall3,
                        '상품명': km_mall4,
                        '가격': km_mall5,
                        'id': km_mall6,
                        'url': km_mall7})

    return km_mall_last_df

# 대중메디팜

In [33]:
# 대중메디팜 // 메디스트림 리스트는 인덱스 채우기용
daejoong_for_marekt_result = medistream_name_list + daejoong_market_name_list
daejoong_for_name_result = medistream_name_list + daejoong_name_list 
daejoong_for_price_result = medistream_name_list + daejoong_price_list
daejoong_for_id_result = medistream_name_list + daejoong_id_list
daejoong_for_url_result = medistream_name_list + daejoong_url_list
daejoong_for_rex_result = medistream_name_list + daejoong_rex_list

# 대중메디팜 인덱스 찾기 위한 딕셔너리
daejoong_index_search = {}

for i in range(len(daejoong_rex_list)):
    daejoong_index_search[daejoong_rex_list[i].strip()] = i

In [34]:
def daejoong_func(medistream_name_input):
    medistream_rex_for_comparison = medistream_rex_dic[medistream_name_input.strip()]

    daejoong1 = []
    daejoong2 = []

    daejoong3 = []
    daejoong4 = []
    daejoong5 = []
    daejoong6 = []
    daejoong7 = []




    len1 = len(medistream_name_list)
    len8 = len(daejoong_name_list)
    # 메디스트림 상품을 제외하고 시작
    for i in range(len1, len1+len8):

        daejoong_result = cos_sim(medistream_daejoong_tfidf_for_sim[medistream_name_input.strip()], medistream_daejoong_tfidf[i])
        if  daejoong_result >= 0.1:
            daejoong1.append(daejoong_for_rex_result[i])
            daejoong2.append(daejoong_result)


        daejoong_df = pd.DataFrame({'정규표현식_상품명': daejoong1,
                            '유사도': daejoong2})
        
        # 쇼핑몰에서 정답 topk개 출력
        daejoong_sort_results = daejoong_df.sort_values(by=['유사도'],ascending=False)

        # topk내에서 2차 매칭
        daejoong_new_topk = {}
        daejoong_val_list = []
        daejoong_topks = list(daejoong_sort_results['정규표현식_상품명'].head(10))

        if len(daejoong_topks) > 0:
            for i in range(len(daejoong_topks)):
                daejoong_new = daejoong_topks[i]
                daejoong_token = twitter.morphs(daejoong_new)
                daejoong_cnt = 0
                daejoong_same = []
                for i in range(len(daejoong_token)):
                    daejoong_comp = (daejoong_token[i], medistream_rex_for_comparison.find(daejoong_token[i]))
                    if daejoong_comp[1] != -1:
                        daejoong_cnt += 1
                        daejoong_same.append(daejoong_token[i])
                # 타사 상품명:일치 개수
                daejoong_new_topk[daejoong_new] = daejoong_cnt

        # daejoong_new_topk에서 daejoong_cnt순으로 top 5뽑기 
        daejoong_sort_new = dict(sorted(daejoong_new_topk.items(), key = lambda item: item[1], reverse = True))
        daejoong_new_list = list(daejoong_sort_new.keys())[:5]

    daejoong_last_name_list = []
    for i in daejoong_new_list:
        daejoong_last_name_list.append(i.strip())


    for i in daejoong_last_name_list:
        daejoong3.append(daejoong_market_name_list[daejoong_index_search[i]])
        daejoong4.append(daejoong_name_list[daejoong_index_search[i]])
        daejoong5.append(daejoong_price_list[daejoong_index_search[i]])
        daejoong6.append(daejoong_id_list[daejoong_index_search[i]])
        daejoong7.append(daejoong_url_list[daejoong_index_search[i]])





    daejoong_last_df = pd.DataFrame({'쇼핑몰': daejoong3,
                        '상품명': daejoong4,
                        '가격': daejoong5,
                        'id': daejoong6,
                        'url': daejoong7})

    return daejoong_last_df

# 결과물 출력

In [35]:
medistream_name_input = input().strip()

medistream_df = pd.DataFrame({'쇼핑몰':'메디스트림',
                     '상품명': medistream_name_input,
                     '가격' : medistream_price_dic[medistream_name_input],
                     'id':medistream_id_dic[medistream_name_input]})

display(medistream_df)
display(akom_func(medistream_name_input))
display(anjin_func(medistream_name_input))
display(hane_func(medistream_name_input))
display(hanpure_func(medistream_name_input))
display(human_hub_func(medistream_name_input))
display(km_mall_func(medistream_name_input))
display(daejoong_func(medistream_name_input))

[진산] 척추모형


,쇼핑몰,상품명,가격,id
$oid,메디스트림,[진산] 척추모형,96000.0,62a02b5a9d938800240730d2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


,쇼핑몰,상품명,가격,id,url
0,아콤몰,척추모형,330000.0,5254d948b166568c2aa9fae3b93da460,https://www.akommall.com/index.php?channel=vie...
1,아콤몰,미니 척추모형(3B),NaN,7c9536d422f2642ccb05d51e9ab8f8e3,https://www.akommall.com/index.php?channel=vie...
2,아콤몰,기본척추모형(XC-105),90000.0,9267aa4ab49f82682b0c4b1ddcfc9c06,https://www.akommall.com/index.php?channel=vie...
3,아콤몰,XC-126A 골반 척추 모형,180000.0,67095ba404269b4d1bdf5c22d8d3121a,https://www.akommall.com/index.php?channel=vie...
4,아콤몰,XC-105C 교육용 골반 척추 모형,150000.0,ad662e8ac68fb65c9c9bab9e9ed8af66,https://www.akommall.com/index.php?channel=vie...


,쇼핑몰,상품명,가격,id,url
0,안진팜메디,"[3B모형] 척추모형 (고급형-근육 부분채색,대퇴골두 분리가능) (모델:A58/7)...",451000.0,2bcc73c5d084f98efac0ec11f5727e12,https://www.anjinmed.com/goods/goods_view.php?...
1,안진팜메디,"[3B모형] 척추모형 (기본형-근육 부분채색,대퇴골 표현) (모델:A58/3) ◈E...",339000.0,47c7824bbc5b4440882ad4b4fbeecfae,https://www.anjinmed.com/goods/goods_view.php?...
2,안진팜메디,[3B모형] 척추변성모형(척추체 및 추간판의 퇴행성 변화 설명) (모델:A795) ...,300000.0,d87c4938336a5351310013efc3c906e3,https://www.anjinmed.com/goods/goods_view.php?...
3,안진팜메디,[3B모형] 견관절모형 (근육모형/회전근개가 있는 운동형 견관절 모형) (모델:A8...,453000.0,ce1da95d6159f610d38935b83531fd5d,https://www.anjinmed.com/goods/goods_view.php?...
4,안진팜메디,[3B모형] 척추모형 (모델:A58/2) ◈중국생산◈ (a0339),256000.0,5abbfb2f5741e8d00220fcc4d8217427,https://www.anjinmed.com/goods/goods_view.php?...


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


,쇼핑몰,상품명,가격,id,url


,쇼핑몰,상품명,가격,id,url
0,케이엠몰,[위엽] 손모형(13cm),5800,9ee1cda29ae2681bf895640579dc614b,https://www.kmmall.net/shop/goods/goods_view.p...
1,케이엠몰,[위엽] 인체모형(男) 26cm,37000,2399b8b7b4ea8288db88e1810a10fa6a,https://www.kmmall.net/shop/goods/goods_view.p...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


,쇼핑몰,상품명,가격,id,url
0,대중메디팜,척추모형(중국제) 82cm,132000.0,62648cd1b46872ca528d8e8752f9710e,https://12oneshop.com/mall/m_mall_detail.php?p...
1,대중메디팜,지성)척추모형,-1.0,8aee622f6d1526784fd35bf029873a10,https://12oneshop.com/mall/m_mall_detail.php?p...
2,대중메디팜,3B 척추모형 A58/2,272500.0,09a8c8b0f6f558e50565dac519d571d4,https://12oneshop.com/mall/m_mall_detail.php?p...
3,대중메디팜,한솔) 귀모형 小,7700.0,835bb01c271f734670ac27fbb8bffe09,https://12oneshop.com/mall/m_mall_detail.php?p...
4,대중메디팜,한솔) 귀모형 大,20000.0,fa931d595ce96ff7117f1814baf4a6f4,https://12oneshop.com/mall/m_mall_detail.php?p...
